# Polar angle

In [1]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd


# figure imports
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px
# from plot_utils import plotly_template
import plotly.express as px

# import data base
main_dir = '/home/mchevillard/disks/meso_shared/gaze_exp/'
pp_dir = "{}/derivatives/pp_data".format(main_dir)
tsv_dir = '{}/sub-all/prf/tsv'.format(pp_dir)

data = pd.read_table('{}/group_task-prf_prf.tsv'.format(tsv_dir))
data = data.drop(columns=['Unnamed: 0'])
data_loo = pd.read_table('{}/group_task-loo_prf_prf.tsv'.format(tsv_dir))
data_loo = data_loo.drop(columns=['Unnamed: 0'])

In [2]:
# define parameters
ecc_th = [0,15]
size_th= [0.1,20]
r2_th = 0.1

# Replace all data outer threshold with NaN data
data.loc[(data.ecc < ecc_th[0]) | (data.ecc > ecc_th[1]) | (data.sd < size_th[0]) | (data.sd > size_th[1]) | (data.r2 <=r2_th)] = np.nan
data = data.dropna()
data_loo.loc[(data_loo.ecc<ecc_th[0]) | (data_loo.ecc > ecc_th[1]) | (data_loo.sd < size_th[0]) | (data_loo.sd > size_th[1]) | (data_loo.r2 <=r2_th)] = np.nan
data_loo = data_loo.dropna()

In [3]:
rois = pd.unique(data.roi)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])
rois_loo = pd.unique(data_loo.roi)
mask_loo = pd.notnull(data_loo.subject)
subjects_loo = pd.unique(data_loo.subject[mask_loo])

In [4]:
# roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

In [5]:
#figures
fig_polar_angle_1 = go.Figure()
fig_polar_angle_2 = go.Figure()
fig_contra_1 = go.Figure()
fig_contra_2 = go.Figure()

# Barpolar for 1 subject and 1 ROI

In [92]:
# Filtrer les données pour V1 et l'hémisphère droit du sujet 1
df = data_loo.loc[(data_loo['subject'] =='sub-001') & (data_loo['roi'] =='V1')]

#Conversion
df.angle = np.degrees(df.angle)
df.angle = np.mod(df.angle, 360)


# Créer les tranches d'angle polaire
num_slices = 12
theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)

# Compter le nombre de voxels dans chaque tranche
voxels_counts, _ = np.histogram(df.angle, bins=theta_slices)

# Créer le graphique barpolar
fig = go.Figure(go.Barpolar(r=voxels_counts,
                            theta=theta_slices,
                            width=30,
                            marker_color=roi_colors_loo[0],
                            marker_line_color='black',  # Couleur de la ligne des barres
                            marker_line_width=1,  # Épaisseur de la ligne des barres
                            opacity=0.8  # Opacité des barres
))

# Define parameters
fig.update_polars(radialaxis=dict(visible=False), angularaxis=dict(visible=False), barmode='overlay')

fig.update_layout(title='Barpolar - Sub-001 - V1 ', title_font_size=20, template='simple_white')

fig.write_image('/home/mchevillard/disks/meso_S/figures/sub-001_polar_V1.pdf')

[152  70  35  46  82 135 255 168 117 125 128 213]


# Subplot all ROIs by subject

In [49]:
rows, cols = 1, 12
fig_height, fig_width = 300, 1920
specs = [[{'type': 'polar'}] * cols]

fig_polar_angle_1 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
fig_polar_angle_2 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_polar_angle_1 if i==0 else fig_polar_angle_2

        df = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi==roi)]
        df = df.sort_values('r2', ascending=False)
        df = df.head(500)
        #Conversion
        df.angle = np.degrees(df.angle)
        df.angle = np.mod(df.angle, 360)

        # Parts of polar angles and number of voxels in each part
        num_slices = 12
        theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)
        voxels_counts, _ = np.histogram(df.angle, bins=theta_slices)

        # barpolar
        fig.add_trace(go.Barpolar(r=voxels_counts, theta=theta_slices, width=30, marker_color=roi_colors_loo[j], marker_line_color='black', marker_line_width=1, opacity=0.8), row=1, col=j+1)

    # Define parameters
    
    fig.update_polars(angularaxis=dict(visible=False), radialaxis=dict(visible=False))
    fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')

    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_subplot_polar.pdf'.format(subject))

## Only RIGHT

In [50]:
rows, cols = 1, 12
fig_height, fig_width = 300, 1920
specs = [[{'type': 'polar'}] * cols]

fig_polar_angle_1 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
fig_polar_angle_2 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_polar_angle_1 if i==0 else fig_polar_angle_2

        df = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi==roi) & (data_loo.hemi == 'R')]
        df = df.sort_values('r2', ascending=False)
        df = df.head(500)
        #Conversion
        df.angle = np.degrees(df.angle)
        df.angle = np.mod(df.angle, 360)

        # Parts of polar angles and number of voxels in each part
        num_slices = 12
        theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)
        voxels_counts, _ = np.histogram(df.angle, bins=theta_slices)

        # barpolar
        fig.add_trace(go.Barpolar(r=voxels_counts, theta=theta_slices, width=30, marker_color=roi_colors_loo[j], marker_line_color='black', marker_line_width=1, opacity=0.8), row=1, col=j+1)

    # Define parameters
    
    fig.update_polars(angularaxis=dict(visible=False), radialaxis=dict(visible=False))
    fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')

    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_subplot_polar_R.pdf'.format(subject))


## Only LEFT

In [47]:
rows, cols = 1, 12
fig_height, fig_width = 300, 1920
specs = [[{'type': 'polar'}] * cols]

fig_polar_angle_1 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
fig_polar_angle_2 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_polar_angle_1 if i==0 else fig_polar_angle_2

        df = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi==roi) & (data_loo.hemi == 'L')]
        df = df.sort_values('r2', ascending=False)
        df = df.head(500)
        #Conversion
        df.angle = np.degrees(df.angle)
        df.angle = np.mod(df.angle, 360)

        # Parts of polar angles and number of voxels in each part
        num_slices = 12
        theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)
        voxels_counts, _ = np.histogram(df.angle, bins=theta_slices)

        # barpolar
        fig.add_trace(go.Barpolar(r=voxels_counts, theta=theta_slices, width=30, marker_color=roi_colors_loo[j], marker_line_color='black', marker_line_width=1, opacity=0.8), row=1, col=j+1)

    # Define parameters
    
    fig.update_polars(angularaxis=dict(visible=False), radialaxis=dict(visible=False))
    fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')

    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_subplot_polar_L.pdf'.format(subject))

# Contralaterality 
## One subject one area

In [20]:
# Filtrer les données pour V1 et l'hémisphère droit du sujet 1
df_right = data_loo.loc[(data_loo['subject'] =='sub-001') & (data_loo['roi'] =='iPCS') & (data_loo['hemi'] == 'R')]

# Filtrer les données pour V1 et l'hémisphère gauche du sujet 1
df_left = data_loo.loc[(data_loo['subject'] =='sub-001') & (data_loo['roi'] =='iPCS') & (data_loo['hemi'] == 'L')]

# Conversion
df_right.angle = np.degrees(df_right.angle)
df_right.angle = np.mod(df_right.angle, 360)

df_left.angle = np.degrees(df_left.angle)
df_left.angle = np.mod(df_left.angle, 360)

# Créer les tranches d'angle polaire
num_slices = 12
theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)

# Compter le nombre de voxels dans chaque tranche pour l'hémisphère droit
voxels_counts_right, _ = np.histogram(df_right.angle, bins=theta_slices)

# Compter le nombre de voxels dans chaque tranche pour l'hémisphère gauche
voxels_counts_left, _ = np.histogram(df_left.angle, bins=theta_slices)

# Calculer le pourcentage de contralatéralité pour chaque hémisphère
percentage_right = sum(voxels_counts_right[3:9]) / sum(voxels_counts_right) * 100
percentage_left = sum(voxels_counts_left[9:] + voxels_counts_left[:3]) / sum(voxels_counts_left) * 100

# Calculer le pourcentage d'excentricité total
percentage_total = (percentage_right + percentage_left) / 2

print("Pourcentage de contralatéralité de l'hémisphère droit : {:.2f}%".format(percentage_right))
print("Pourcentage de contralatéralité de l'hémisphère gauche : {:.2f}%".format(percentage_left))
print("Pourcentage de contralatéralité total : {:.1f}%".format(percentage_total))

Pourcentage de contralatéralité de l'hémisphère droit : 31.31%
Pourcentage de contralatéralité de l'hémisphère gauche : 66.21%
Pourcentage de contralatéralité total : 48.8%


In [23]:
# Valeurs correspondant au pourcentage total et au complément à 100%
values = [percentage_total, 100-percentage_total]
fig = go.Figure()


fig.add_trace(go.Pie(values=values,
                     hole=0.5,
                     marker=dict(colors=[roi_colors_loo[9], 'rgb(255,255,255)'], line=dict(color=['black', 'white'], width=[1,0])),
                     showlegend=False,
                     textfont=dict(color=['black', 'white']),
                     textfont_size=12, 
                     pull=[0, 0.05]
                     ))

# Mise en forme du layout
fig.update_layout(showlegend=False, template='simple_white')


fig.write_image('/home/mchevillard/disks/meso_S/figures/sub-001_contra_iPCS.pdf')


## Subplot all areas

In [43]:
rows, cols = 1, 12
fig_height, fig_width = 300, 1920
specs = [[{'type': 'pie'}] * cols]

fig_contra_1 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
fig_contra_2 = make_subplots(rows=rows, cols=cols, print_grid=False, specs=specs)
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_contra_1 if i==0 else fig_contra_2

        
        df_right = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi == roi) & (data_loo.hemi == 'R')]
        df_left = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi == roi) & (data_loo.hemi == 'L')]
        df_right = df_right.sort_values('r2', ascending=False)
        df_right = df_right.head(500)
        df_left = df_left.sort_values('r2', ascending=False)
        df_left = df_left.head(500)
        
        # Conversion
        df_right.angle = np.degrees(df_right.angle)
        df_right.angle = np.mod(df_right.angle, 360)

        df_left.angle = np.degrees(df_left.angle)
        df_left.angle = np.mod(df_left.angle, 360)

        # Créer les tranches d'angle polaire
        num_slices = 12
        theta_slices = np.linspace(0, 360, num_slices+1, endpoint=True)

        # Compter le nombre de voxels dans chaque tranche pour l'hémisphère droit
        voxels_counts_right, _ = np.histogram(df_right.angle, bins=theta_slices)

        # Compter le nombre de voxels dans chaque tranche pour l'hémisphère gauche
        voxels_counts_left, _ = np.histogram(df_left.angle, bins=theta_slices)

        # Calculer le pourcentage de contralatéralité pour chaque hémisphère
        percentage_right = sum(voxels_counts_right[3:9]) / sum(voxels_counts_right) * 100
        percentage_left = sum(voxels_counts_left[9:] + voxels_counts_left[:3]) / sum(voxels_counts_left) * 100

        # Calculer le pourcentage d'excentricité total
        percentage_total = (percentage_right + percentage_left) / 2

        print("{} - Contralateralité in {}: {:.1f}%".format(subject, roi, percentage_total))
        percentage_rest = 100 - percentage_total        
        
        values = [percentage_total, percentage_rest]
        
        fig.add_trace(go.Pie(values=values,
                             hole=0.5,
                             marker=dict(colors=[roi_colors_loo[j], 'white'], line=dict(color=['black', 'white'], width=[1,0])),
                             showlegend=False,
                             textfont=dict(color=['black', 'white']),
                             textfont_size=14, 
                             pull=[0, 0.01],
                             rotation=180 if percentage_total < percentage_rest else 0,
                             direction='clockwise' if percentage_total < percentage_rest else 'counterclockwise'
                             ), row=1, col=j+1)

    # Define parameters
    fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')

    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_contralaterality.pdf'.format(subject))
    

sub-001 - Contralateralité in V1: 76.9%
sub-001 - Contralateralité in V2: 70.6%
sub-001 - Contralateralité in V3: 63.3%
sub-001 - Contralateralité in V3AB: 75.2%
sub-001 - Contralateralité in LO: 75.0%
sub-001 - Contralateralité in VO: 77.1%
sub-001 - Contralateralité in hMT+: 67.8%
sub-001 - Contralateralité in iIPS: 60.4%
sub-001 - Contralateralité in sIPS: 70.1%
sub-001 - Contralateralité in iPCS: 49.2%
sub-001 - Contralateralité in sPCS: 59.0%
sub-001 - Contralateralité in mPCS: 49.6%
sub-002 - Contralateralité in V1: 78.5%
sub-002 - Contralateralité in V2: 61.9%
sub-002 - Contralateralité in V3: 70.8%
sub-002 - Contralateralité in V3AB: 68.1%
sub-002 - Contralateralité in LO: 72.7%
sub-002 - Contralateralité in VO: 78.9%
sub-002 - Contralateralité in hMT+: 75.3%
sub-002 - Contralateralité in iIPS: 59.4%
sub-002 - Contralateralité in sIPS: 55.9%
sub-002 - Contralateralité in iPCS: 56.8%
sub-002 - Contralateralité in sPCS: 48.1%
sub-002 - Contralateralité in mPCS: 57.6%
